In [5]:
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

<h4>Step1: Dataset</h4>

In [11]:
# Check for GPU availability and set memory growth
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("GPU is available")
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    print("No GPU found, using CPU")



import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print(f"CUDA is available. Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("CUDA is not available.")


No GPU found, using CPU
CUDA is not available.


In [4]:
# Define dataset directory
data_dir = './dataset/asl_alphabet_train/asl_alphabet_train/'

# Initialize lists to hold data and labels
data = []
labels = []

# Set the desired image dimensions
IMG_SIZE = 224  # Adjust as needed

<h4>Step2: Pre-process the data</h4>

In [6]:
# Function to handle missing or corrupted images
def is_valid_image(img_path):
    try:
        img = cv2.imread(img_path)
        if img is None:
            return False  # File is corrupted or unreadable
        return True
    except:
        return False

In [10]:
def process_image(file_path, folder):
    if is_valid_image(file_path):
        image = cv2.imread(file_path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = image.astype('float32') / 255.0
        return image, folder
    else:
        print(f"Warning: Skipping corrupted image {file_path}")
        return None, None

def load_images(data_dir):
    data, labels = [], []
    with ThreadPoolExecutor() as executor:
        futures = []
        for folder in os.listdir(data_dir):
            folder_path = os.path.join(data_dir, folder)
            if os.path.isdir(folder_path):
                for file in os.listdir(folder_path):
                    file_path = os.path.join(folder_path, file)
                    futures.append(executor.submit(process_image, file_path, folder))
        
        # Collect results
        for future in futures:
            image, label = future.result()
            if image is not None:
                data.append(image)
                labels.append(label)

    return data, labels

In [ ]:
#pre-process the data concurrent
data, labels = load_images(data_dir)

In [ ]:
# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

In [ ]:
# Handle potential missing labels (if there are any)
valid_data_indices = ~np.isnan(labels)  # If labels are numeric, adapt if needed
data = data[valid_data_indices]
labels = labels[valid_data_indices]

In [ ]:
# One-hot encode labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
# Split dataset into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# Summary of dataset shape
print(f"Training set: {X_train.shape}, Validation set: {X_val.shape}, Test set: {X_test.shape}")

<h4>Step3: Data Exploration</h4>

In [ ]:
# 1. Check distribution of classes (labels)
unique, counts = np.unique(labels, return_counts=True)
class_distribution = dict(zip(unique, counts))

plt.figure(figsize=(12,6))
sns.barplot(x=list(class_distribution.keys()), y=list(class_distribution.values()))
plt.title('Class Distribution (ASL Letters)')
plt.xlabel('Classes')
plt.ylabel('Frequency')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# 2. Visualize sample images from each class
def visualize_samples_per_class(data, labels, num_samples=5):
    unique_classes = np.unique(labels)
    plt.figure(figsize=(12,12))
    
    for i, label in enumerate(unique_classes):
        class_indices = np.where(np.argmax(labels, axis=1) == i)[0]
        sample_images = data[class_indices][:num_samples]
        
        for j in range(num_samples):
            plt.subplot(len(unique_classes), num_samples, i * num_samples + j + 1)
            plt.imshow(sample_images[j])
            plt.title(f"Class: {label}")
            plt.axis('off')

    plt.show()

visualize_samples_per_class(X_train, y_train, num_samples=3)

In [ ]:
# 3. Analyze pixel value distributions for all images (are they normally distributed?)
flattened_images = X_train.reshape(X_train.shape[0], -1)

plt.figure(figsize=(10,6))
sns.histplot(flattened_images.flatten(), bins=50, kde=True)
plt.title('Pixel Value Distribution')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# 4. Check for outliers in pixel intensities (before normalization)
plt.figure(figsize=(10,6))
sns.boxplot(x=flattened_images.flatten())
plt.title('Pixel Value Outliers (Boxplot)')
plt.show()

In [ ]:
# 5. Visualizing correlation of pixel positions (Optional but advanced for image analysis)
# Here you can use methods like PCA or t-SNE for dimensionality reduction to visualize correlations
from sklearn.decomposition import PCA

# Reshape the data for PCA (flattening images)
X_train_flat = X_train.reshape(X_train.shape[0], -1)

# Apply PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_train_flat)

plt.figure(figsize=(10, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=np.argmax(y_train, axis=1), cmap='tab10', alpha=0.6)
plt.colorbar()
plt.title('PCA of ASL Alphabet Dataset')
plt.show()

In [ ]:
# Summary statistics (mean, std, etc.)
print(f"Mean pixel value: {np.mean(flattened_images)}")
print(f"Median pixel value: {np.median(flattened_images)}")
print(f"Standard deviation of pixel values: {np.std(flattened_images)}")

<h4>Step4: Model Building</h4>